# Date Binning with the `Period` Class

The following code demonstrates various ways to use and apply the functionality in the `Period` class. In general, the class expects a date range defined with a `start_date` and `end_date`, and a `periodicity` (see options below). From there, it can generate date bins from `start_date` to `end_date` (inclusive by default) for the given `periodicity`. Date bins are in the form of a list of tuples, where each tuple contains that bin's start date and end date.

The class can also convert given date bins from one periodicity to another, it can redistribute data from one periodicity to another given the data and original date bins, and can output pre-defined or custom labels.

The class supports the following options for `periodicity`. The ISO options follow the ISO-8601 Standard and enforce a Monday week start - if a given `start_date` is not a Monday, there are side effects detailed in the "Partial ('Stub') Periods" section. The "Fiscal Weeks" `periodicity` may be a better option when Monday is not the week start. If a `periodicity` isn't provided, the default is "ISO Week".

- "ISO Week" (default): weekly bins starting on a Monday
- "ISO Biweekly": bins of 2 ISO week periods, grouping weeks 01 and 02, then 03 with 04, etc.
- "ISO Month (4 Weeks)": monthly bins where months are defined by grouping ISO weeks within the ISO year in a pattern of 4 weeks each (this creates 13 months). Month 1 is weeks 01-04, month 2 is weeks 05-08, etc.
- "ISO Month (4 + 5 + 4)": monthly bins where months are defined by grouping ISO weeks within the ISO year in a pattern of 4 weeks, 5 weeks, then 4 weeks. Month 1 is weeks 01-04, month 2 is weeks 05-09, etc.
- "ISO Month (4 + 4 + 5)": monthly bins where months are defined by grouping ISO weeks within the ISO year in a pattern of 4 weeks, 4 weeks, then 5 weeks. Month 1 is weeks 01-04, month 2 is weeks 05-08, etc.
- "ISO Quarter (13 Weeks)": quarterly bins of 13 ISO week periods. Quarter 1 is weeks 01-13, quarter 2 is weeks 14-26, etc.
- "ISO Semiannual (26 Weeks)": semiannual bins of 26 ISO week periods. The first period is weeks 01-26 and second is weeks 27 to either 52 or 53, depending on the year
- "ISO Annual": bins of full ISO years. If `start_date` is not the first day of the ISO year, the first bin will be a partial year
- "Custom Days": bins starting on `start_date` with a number of days between them as given by the `custom_period` parameter in `get_date_bins` (this may be a single integer or a sequence that's cycled over)
- "Weekly": weekly bins starting on `start_date`'s weekday
- "Biweekly": bins of 2-week periods, starting on `start_date`'s weekday
- "Fiscal Weeks": bins starting on `start_date` with a number of weeks between them as given by the `custom_period` parameter in `get_date_bins` (this may be a single integer, like 13 weeks, or a sequence such as [4, 5, 4]). This option is similar to the ISO patterns, but counts weeks starting with the given `start_date`, regardless of which day of the week it is
- "Calendar Month": bins by calendar month
- "Monthly": monthly bins starting on `start_date`
- "Calendar Quarter": bins of 3-month periods based on a calendar year (Jan-Mar, Apr-Jun, etc.)
- "Quarterly": bins of 3-month periods starting on `start_date`
- "Calendar Year": calendar year bins (Jan-Dec). If `start_date` is not Jan 1, the first bin will be a partial year
- "Annually": yearly bins starting from `start_date`
- "Entire Period": one bin spanning the given date range

## Partial ("Stub") Periods

The `Period` class respects the user-provided `start_date` and `end_date`. Certain `periodicity` options, such as the ISO ones, adhere to standards, and others, such as the "Calendar"-based ones, follow convention. When user inputs don't align with those standards or conventions, then `get_date_bins` may include a partial, or "stub" period at the beginning or end of the date bins.

For example, if the user wants bins by "Calendar Year", but the `start_date` they provide is not January 1st, then the first bin will be a stub year from `start_date` to December 31st.

For all ISO periodicity options, if `start_date` is not a Monday or week 01 of any other ISO period, the first bin will represent a stub period.

For periodicity options of "Calendar Month", "Calendar Quarter", and "Calendar Year", if `start_date` is not the 1st of the period, the first bin will represent a partial month/quarter/year. "Calendar Quarters" are defined as January 31st to March 31st, April 30th to June 30th, July 31st to September 30th, and October 31st to December 31st.

Not all `periodicity` choices create stub periods. For the "Custom Days", "Weekly", "Biweekly", "Fiscal Weeks", "Monthly", "Quarterly", and "Annually" options, there's no concept of a partial period - `get_date_bins` starts binning with `start_date` and generates the bins from there.

In [1]:
import datetime
from dateutil.relativedelta import relativedelta

from forecast import Period

## Using the `Period` Class


### Get Date Bins for ISO Week Periodicity

The following example creates date bins by ISO week for the current year.

The default `periodicity` for the `Period` class is "ISO Week", so it's not necessary to provide it. Also, `get_date_bins` has a parameter `inclusive` that by default is `True` - this means the given `end_date` is included in the bins. If it's set to `False`, then the bins will end the day before the given `end_date`, as the example below shows.

In [4]:
d = Period().get_date_bins(start_date, end_date, "ISO Week", inclusive=False)

In [6]:
current_year = datetime.datetime.now().year
start_date = datetime.date.fromisocalendar(current_year, 1, 1)
end_date = datetime.date.fromisocalendar(current_year + 1, 1, 1)

# Create an instance of Period and generate the date bins
p = Period(start_date=start_date, end_date=end_date, periodicity="ISO Week")
bins = p.get_date_bins(inclusive=False)

# Alternative way to call the class and method:
# bins = Period().get_date_bins(start_date, end_date, "ISO Week", inclusive=False)

bins

[(datetime.date(2024, 12, 30), datetime.date(2025, 1, 5)),
 (datetime.date(2025, 1, 6), datetime.date(2025, 1, 12)),
 (datetime.date(2025, 1, 13), datetime.date(2025, 1, 19)),
 (datetime.date(2025, 1, 20), datetime.date(2025, 1, 26)),
 (datetime.date(2025, 1, 27), datetime.date(2025, 2, 2)),
 (datetime.date(2025, 2, 3), datetime.date(2025, 2, 9)),
 (datetime.date(2025, 2, 10), datetime.date(2025, 2, 16)),
 (datetime.date(2025, 2, 17), datetime.date(2025, 2, 23)),
 (datetime.date(2025, 2, 24), datetime.date(2025, 3, 2)),
 (datetime.date(2025, 3, 3), datetime.date(2025, 3, 9)),
 (datetime.date(2025, 3, 10), datetime.date(2025, 3, 16)),
 (datetime.date(2025, 3, 17), datetime.date(2025, 3, 23)),
 (datetime.date(2025, 3, 24), datetime.date(2025, 3, 30)),
 (datetime.date(2025, 3, 31), datetime.date(2025, 4, 6)),
 (datetime.date(2025, 4, 7), datetime.date(2025, 4, 13)),
 (datetime.date(2025, 4, 14), datetime.date(2025, 4, 20)),
 (datetime.date(2025, 4, 21), datetime.date(2025, 4, 27)),
 (date

If `start_date` is not a Monday, or `end_date` is not a Sunday, the first or last bin will be a stub period, respectively. The following example uses a Wednesday (weekday 3) in week 01 as `start_date` and a Friday (weekday 5) in week 04 as an `end_date`. The resulting first and last bins are partial weeks to respect the user's date range. The middle 2 bins are full ISO weeks starting on Monday.

In [7]:
start_date = datetime.date.fromisocalendar(current_year, 1, 3)
end_date = datetime.date.fromisocalendar(current_year, 4, 5)

# Create an instance of Period and generate the date bins
p = Period(start_date=start_date, end_date=end_date)
bins = p.get_date_bins()
bins

[(datetime.date(2025, 1, 1), datetime.date(2025, 1, 5)),
 (datetime.date(2025, 1, 6), datetime.date(2025, 1, 12)),
 (datetime.date(2025, 1, 13), datetime.date(2025, 1, 19)),
 (datetime.date(2025, 1, 20), datetime.date(2025, 1, 24))]

### Get Labels for Date Bins

The `Period` class has the `get_period_labels` method to generate labels for given date bins. It has built-in formats for all periodicity options, or the user can provide their own format string to suit their needs. The method supports any Python [strftime formatters](https://docs.python.org/3/library/datetime.html#strftime-and-strptime-format-codes), which uses the 1989 C standard.

If the user provides a custom format, they can also indicate which date for each bin - the bin's start date or the bin's end date - to use to create the labels with the `use_bin_start_date_for_label` parameter. If the user doesn't provide this, the built-in formats use the bin's start date for ISO-based periodicity options and the bin's end date for the others, with the exception of the "Entire Period" periodicity, which uses both dates.

In [10]:
# Get default-formatted labels
p.get_period_labels(bins, periodicity="ISO Week")

['Week 1-25', 'Week 2-25', 'Week 3-25', 'Week 4-25']

In [11]:
# Get custom-formatted labels of form MMM-DD using the end date of each bin
p.get_period_labels(
    bins,
    periodicity="ISO Week",
    date_format_string="%b-%d",
    use_bin_start_date_for_label=False
)

['Jan-05', 'Jan-12', 'Jan-19', 'Jan-24']

### Convert Date Bins from One Periodicity to Another

The following example converts bins from "ISO Week" to "ISO Quarter (13 Weeks)".

In [12]:
start_date = datetime.date.fromisocalendar(current_year, 1, 1)
end_date = datetime.date.fromisocalendar(current_year + 1, 1, 1) - relativedelta(days=1)

# Create an instance of Period and generate the date bins
p = Period(start_date=start_date, end_date=end_date)
iso_week_bins = p.get_date_bins()

# Convert ISO Week bins to ISO Quarter (13 Weeks) bins
iso_q_bins = p.convert_dates(iso_week_bins, "ISO Quarter (13 Weeks)")
iso_q_bins

[(datetime.date(2024, 12, 30), datetime.date(2025, 3, 30)),
 (datetime.date(2025, 3, 31), datetime.date(2025, 6, 29)),
 (datetime.date(2025, 6, 30), datetime.date(2025, 9, 28)),
 (datetime.date(2025, 9, 29), datetime.date(2025, 12, 28))]

In [13]:
p.get_period_labels(iso_q_bins, "ISO Quarter (13 Weeks)")

['Q1-25', 'Q2-25', 'Q3-25', 'Q4-25']

### Redistribute Data from One Bin to Another

Given numerical data and date bins of the same length, the `Period` class's `redistribute_data` method can convert the date bins and "re-bin" the numerical data values. Note that it assumes the data are distributed evenly across the days in their respective bins before they are redistributed to the new bins.

The resulting object has the new date bins as keys and the redistributed data as values.

In [14]:
n_weeks_in_cy = datetime.date(current_year, 12, 28).isocalendar().week
data = [100] * 13 + [200] * 13 + [300] * 13 + [100] * 13
data = data + ([100] if n_weeks_in_cy == 53 else [])

# Convert ISO Week data to ISO Quarter data
q_data = p.redistribute_data(data, iso_week_bins, "ISO Quarter (13 Weeks)")
q_data

OrderedDict([((datetime.date(2024, 12, 30), datetime.date(2025, 3, 30)),
              Decimal('1299.999999999999999999999995')),
             ((datetime.date(2025, 3, 31), datetime.date(2025, 6, 29)),
              Decimal('2600.000000000000000000000025')),
             ((datetime.date(2025, 6, 30), datetime.date(2025, 9, 28)),
              Decimal('3900.000000000000000000000010')),
             ((datetime.date(2025, 9, 29), datetime.date(2025, 12, 28)),
              Decimal('1299.999999999999999999999995'))])

## ISO Periodicity Date Bin Examples

The following examples illustrate the other ISO-based periodicity options for creating date bins. All examples use a `start_date` that is the last week of the prior ISO year and an `end_date` in the first week of the following ISO year. This is to show that the binning applies patterns of including specific ISO weeks in a defined "month", "quarter", half year, etc.

### ISO Biweekly

Note that the first bin is only 1 week, since it's a stub period of the last biweekly period in the prior ISO year, followed by 2-week bins that group ISO weeks 1 and 2, then 3 and 4, and so forth. The final bin is also a 1 week long stub period since the `end_date` splits that biweekly period.

In [15]:
# `start_date` is the last week of the prior ISO year
start_date = datetime.date.fromisocalendar(current_year, 1, 1) - relativedelta(days=7)

# `end_date` is the first week of the following ISO year
end_date = datetime.date.fromisocalendar(current_year, n_weeks_in_cy, 7) + relativedelta(days=7)
print(f"Start Date: {start_date}, End Date: {end_date}")

Start Date: 2024-12-23, End Date: 2026-01-04


In [16]:
p = Period(start_date=start_date, end_date=end_date, periodicity="ISO Biweekly")
bins = p.get_date_bins()
bins

[(datetime.date(2024, 12, 23), datetime.date(2024, 12, 29)),
 (datetime.date(2024, 12, 30), datetime.date(2025, 1, 12)),
 (datetime.date(2025, 1, 13), datetime.date(2025, 1, 26)),
 (datetime.date(2025, 1, 27), datetime.date(2025, 2, 9)),
 (datetime.date(2025, 2, 10), datetime.date(2025, 2, 23)),
 (datetime.date(2025, 2, 24), datetime.date(2025, 3, 9)),
 (datetime.date(2025, 3, 10), datetime.date(2025, 3, 23)),
 (datetime.date(2025, 3, 24), datetime.date(2025, 4, 6)),
 (datetime.date(2025, 4, 7), datetime.date(2025, 4, 20)),
 (datetime.date(2025, 4, 21), datetime.date(2025, 5, 4)),
 (datetime.date(2025, 5, 5), datetime.date(2025, 5, 18)),
 (datetime.date(2025, 5, 19), datetime.date(2025, 6, 1)),
 (datetime.date(2025, 6, 2), datetime.date(2025, 6, 15)),
 (datetime.date(2025, 6, 16), datetime.date(2025, 6, 29)),
 (datetime.date(2025, 6, 30), datetime.date(2025, 7, 13)),
 (datetime.date(2025, 7, 14), datetime.date(2025, 7, 27)),
 (datetime.date(2025, 7, 28), datetime.date(2025, 8, 10)),
 (

In [17]:
# Get default-formatted labels for date bins
p.get_period_labels(bins, "ISO Biweekly")

['Weeks 52-52 24',
 'Weeks 1-2 25',
 'Weeks 3-4 25',
 'Weeks 5-6 25',
 'Weeks 7-8 25',
 'Weeks 9-10 25',
 'Weeks 11-12 25',
 'Weeks 13-14 25',
 'Weeks 15-16 25',
 'Weeks 17-18 25',
 'Weeks 19-20 25',
 'Weeks 21-22 25',
 'Weeks 23-24 25',
 'Weeks 25-26 25',
 'Weeks 27-28 25',
 'Weeks 29-30 25',
 'Weeks 31-32 25',
 'Weeks 33-34 25',
 'Weeks 35-36 25',
 'Weeks 37-38 25',
 'Weeks 39-40 25',
 'Weeks 41-42 25',
 'Weeks 43-44 25',
 'Weeks 45-46 25',
 'Weeks 47-48 25',
 'Weeks 49-50 25',
 'Weeks 51-52 25',
 'Weeks 1-1 26']

### ISO Month (4 Weeks)

This periodicity defines months as equal, 4-ISO week periods for a total of 13 months. The default labels use "M13" for the final month's name. In the event the ISO year has 53 weeks, the final week is included in the last month.

In [18]:
p = Period(start_date=start_date, end_date=end_date, periodicity="ISO Month (4 Weeks)")
bins = p.get_date_bins()
bins

[(datetime.date(2024, 12, 23), datetime.date(2024, 12, 29)),
 (datetime.date(2024, 12, 30), datetime.date(2025, 1, 26)),
 (datetime.date(2025, 1, 27), datetime.date(2025, 2, 23)),
 (datetime.date(2025, 2, 24), datetime.date(2025, 3, 23)),
 (datetime.date(2025, 3, 24), datetime.date(2025, 4, 20)),
 (datetime.date(2025, 4, 21), datetime.date(2025, 5, 18)),
 (datetime.date(2025, 5, 19), datetime.date(2025, 6, 15)),
 (datetime.date(2025, 6, 16), datetime.date(2025, 7, 13)),
 (datetime.date(2025, 7, 14), datetime.date(2025, 8, 10)),
 (datetime.date(2025, 8, 11), datetime.date(2025, 9, 7)),
 (datetime.date(2025, 9, 8), datetime.date(2025, 10, 5)),
 (datetime.date(2025, 10, 6), datetime.date(2025, 11, 2)),
 (datetime.date(2025, 11, 3), datetime.date(2025, 11, 30)),
 (datetime.date(2025, 12, 1), datetime.date(2025, 12, 28)),
 (datetime.date(2025, 12, 29), datetime.date(2026, 1, 4))]

In [19]:
# Get default-formatted labels for date bins
p.get_period_labels(bins, "ISO Month (4 Weeks)")

['M13 (4w)-24',
 'Jan (4w)-25',
 'Feb (4w)-25',
 'Mar (4w)-25',
 'Apr (4w)-25',
 'May (4w)-25',
 'Jun (4w)-25',
 'Jul (4w)-25',
 'Aug (4w)-25',
 'Sep (4w)-25',
 'Oct (4w)-25',
 'Nov (4w)-25',
 'Dec (4w)-25',
 'M13 (4w)-25',
 'Jan (4w)-26']

### ISO Month (4 + 5 + 4)

This periodicity defines months as grouped ISO weeks in the pattern of 4 weeks, 5 weeks, then 4 weeks, with the first month including ISO weeks 01-04 and so forth. In the event the ISO year has 53 weeks, the final week is included in the last month.

In [20]:
p = Period(start_date=start_date, end_date=end_date, periodicity="ISO Month (4 + 5 + 4)")
bins = p.get_date_bins()
bins

[(datetime.date(2024, 12, 23), datetime.date(2024, 12, 29)),
 (datetime.date(2024, 12, 30), datetime.date(2025, 1, 26)),
 (datetime.date(2025, 1, 27), datetime.date(2025, 3, 2)),
 (datetime.date(2025, 3, 3), datetime.date(2025, 3, 30)),
 (datetime.date(2025, 3, 31), datetime.date(2025, 4, 27)),
 (datetime.date(2025, 4, 28), datetime.date(2025, 6, 1)),
 (datetime.date(2025, 6, 2), datetime.date(2025, 6, 29)),
 (datetime.date(2025, 6, 30), datetime.date(2025, 7, 27)),
 (datetime.date(2025, 7, 28), datetime.date(2025, 8, 31)),
 (datetime.date(2025, 9, 1), datetime.date(2025, 9, 28)),
 (datetime.date(2025, 9, 29), datetime.date(2025, 10, 26)),
 (datetime.date(2025, 10, 27), datetime.date(2025, 11, 30)),
 (datetime.date(2025, 12, 1), datetime.date(2025, 12, 28)),
 (datetime.date(2025, 12, 29), datetime.date(2026, 1, 4))]

In [21]:
# Get default-formatted labels for date bins
p.get_period_labels(bins, "ISO Month (4 + 5 + 4)")

['Dec (4w)-24',
 'Jan (4w)-25',
 'Feb (5w)-25',
 'Mar (4w)-25',
 'Apr (4w)-25',
 'May (5w)-25',
 'Jun (4w)-25',
 'Jul (4w)-25',
 'Aug (5w)-25',
 'Sep (4w)-25',
 'Oct (4w)-25',
 'Nov (5w)-25',
 'Dec (4w)-25',
 'Jan (4w)-26']

### ISO Month (4 + 4 + 5)

This periodicity defines months as grouped ISO weeks in the pattern of 4 weeks, 4 weeks, then 5 weeks, with the first month including ISO weeks 01-04 and so forth. In the event the ISO year has 53 weeks, the final week is included in the last month.

In [22]:
p = Period(start_date=start_date, end_date=end_date, periodicity="ISO Month (4 + 4 + 5)")
bins = p.get_date_bins()
bins

[(datetime.date(2024, 12, 23), datetime.date(2024, 12, 29)),
 (datetime.date(2024, 12, 30), datetime.date(2025, 1, 26)),
 (datetime.date(2025, 1, 27), datetime.date(2025, 2, 23)),
 (datetime.date(2025, 2, 24), datetime.date(2025, 3, 30)),
 (datetime.date(2025, 3, 31), datetime.date(2025, 4, 27)),
 (datetime.date(2025, 4, 28), datetime.date(2025, 5, 25)),
 (datetime.date(2025, 5, 26), datetime.date(2025, 6, 29)),
 (datetime.date(2025, 6, 30), datetime.date(2025, 7, 27)),
 (datetime.date(2025, 7, 28), datetime.date(2025, 8, 24)),
 (datetime.date(2025, 8, 25), datetime.date(2025, 9, 28)),
 (datetime.date(2025, 9, 29), datetime.date(2025, 10, 26)),
 (datetime.date(2025, 10, 27), datetime.date(2025, 11, 23)),
 (datetime.date(2025, 11, 24), datetime.date(2025, 12, 28)),
 (datetime.date(2025, 12, 29), datetime.date(2026, 1, 4))]

In [23]:
# Get default-formatted labels for date bins
p.get_period_labels(bins, "ISO Month (4 + 4 + 5)")

['Dec (5w)-24',
 'Jan (4w)-25',
 'Feb (4w)-25',
 'Mar (5w)-25',
 'Apr (4w)-25',
 'May (4w)-25',
 'Jun (5w)-25',
 'Jul (4w)-25',
 'Aug (4w)-25',
 'Sep (5w)-25',
 'Oct (4w)-25',
 'Nov (4w)-25',
 'Dec (5w)-25',
 'Jan (4w)-26']

### ISO Quarter (13 Weeks)

This periodicity defines quarters as grouped ISO weeks in the pattern of 13 weeks each, with the first quarter including ISO weeks 01-13 and so forth. In the event the ISO year has 53 weeks, the final week is included in the last quarter.

In [24]:
p = Period(start_date=start_date, end_date=end_date, periodicity="ISO Quarter (13 Weeks)")
bins = p.get_date_bins()
bins

[(datetime.date(2024, 12, 23), datetime.date(2024, 12, 29)),
 (datetime.date(2024, 12, 30), datetime.date(2025, 3, 30)),
 (datetime.date(2025, 3, 31), datetime.date(2025, 6, 29)),
 (datetime.date(2025, 6, 30), datetime.date(2025, 9, 28)),
 (datetime.date(2025, 9, 29), datetime.date(2025, 12, 28)),
 (datetime.date(2025, 12, 29), datetime.date(2026, 1, 4))]

In [25]:
# Get default-formatted labels for date bins
p.get_period_labels(bins, "ISO Quarter (13 Weeks)")

['Q4-24', 'Q1-25', 'Q2-25', 'Q3-25', 'Q4-25', 'Q1-26']

### ISO Semiannual (26 Weeks)

This periodicity defines a semi-annual period as grouped ISO weeks in the pattern of 26 weeks each, with the first half-year including ISO weeks 01-26 and the second half-year including weeks 27-52. In the event the ISO year has 53 weeks, the final week is included in the second half-year.

In [26]:
p = Period(start_date=start_date, end_date=end_date, periodicity="ISO Semiannual (26 Weeks)")
bins = p.get_date_bins()
bins

[(datetime.date(2024, 12, 23), datetime.date(2024, 12, 29)),
 (datetime.date(2024, 12, 30), datetime.date(2025, 6, 29)),
 (datetime.date(2025, 6, 30), datetime.date(2025, 12, 28)),
 (datetime.date(2025, 12, 29), datetime.date(2026, 1, 4))]

In [27]:
# Get default-formatted labels for date bins
p.get_period_labels(bins, "ISO Semiannual (26 Weeks)")

['HY2-24', 'HY1-25', 'HY2-25', 'HY1-26']

### ISO Annual

This periodicity returns bins by ISO year.

In [28]:
p = Period(start_date=start_date, end_date=end_date, periodicity="ISO Annual")
bins = p.get_date_bins()
bins

[(datetime.date(2024, 12, 23), datetime.date(2024, 12, 29)),
 (datetime.date(2024, 12, 30), datetime.date(2025, 12, 28)),
 (datetime.date(2025, 12, 29), datetime.date(2026, 1, 4))]

In [29]:
# Get default-formatted labels for date bins
p.get_period_labels(bins, "ISO Annual")

['2024', '2025', '2026']

## Calendar and Other Periodicity Date Bin Examples

The following examples illustrate the calendar-based and other periodicity options for creating date bins.

### Custom Days

The most flexible of the periodicity options, this one creates bins starting on `start_date` with a number of days between them as given by the additional `custom_period` parameter. This may also be a sequence, such as `[5, 2]`, if used with a Monday `start_date`, would create bins of weekdays, then weekends.

In [31]:
start_date = datetime.datetime.now().date()
end_date = start_date + relativedelta(days=30)

p = Period(start_date=start_date, end_date=end_date, periodicity="Custom Days")
bins = p.get_date_bins(custom_period=5)
bins

[(datetime.date(2025, 2, 23), datetime.date(2025, 2, 27)),
 (datetime.date(2025, 2, 28), datetime.date(2025, 3, 4)),
 (datetime.date(2025, 3, 5), datetime.date(2025, 3, 9)),
 (datetime.date(2025, 3, 10), datetime.date(2025, 3, 14)),
 (datetime.date(2025, 3, 15), datetime.date(2025, 3, 19)),
 (datetime.date(2025, 3, 20), datetime.date(2025, 3, 24)),
 (datetime.date(2025, 3, 25), datetime.date(2025, 3, 25))]

In [32]:
p.get_period_labels(bins, "Custom Days")

['02/27/25',
 '03/04/25',
 '03/09/25',
 '03/14/25',
 '03/19/25',
 '03/24/25',
 '03/25/25']

In [35]:
# Use a Monday start_date
start_date = datetime.date.fromisocalendar(current_year, 1, 1)
end_date = start_date + relativedelta(days=13)

p = Period(start_date=start_date, end_date=end_date, periodicity="Custom Days")
bins = p.get_date_bins(custom_period=[5, 2])  # bin by weekday, then weekend days
bins

[(datetime.date(2024, 12, 30), datetime.date(2025, 1, 3)),
 (datetime.date(2025, 1, 4), datetime.date(2025, 1, 5)),
 (datetime.date(2025, 1, 6), datetime.date(2025, 1, 10)),
 (datetime.date(2025, 1, 11), datetime.date(2025, 1, 12))]

### Weekly

This periodicity option creates weekly bins based on `start_date`'s weekday.

In [36]:
start_date = datetime.date(current_year, 1, 15)
end_date = start_date + relativedelta(days=41)

p = Period(start_date=start_date, end_date=end_date, periodicity="Weekly")
bins = p.get_date_bins()
bins

[(datetime.date(2025, 1, 15), datetime.date(2025, 1, 21)),
 (datetime.date(2025, 1, 22), datetime.date(2025, 1, 28)),
 (datetime.date(2025, 1, 29), datetime.date(2025, 2, 4)),
 (datetime.date(2025, 2, 5), datetime.date(2025, 2, 11)),
 (datetime.date(2025, 2, 12), datetime.date(2025, 2, 18)),
 (datetime.date(2025, 2, 19), datetime.date(2025, 2, 25))]

In [37]:
p.get_period_labels(bins, "Weekly")

['01/21/25', '01/28/25', '02/04/25', '02/11/25', '02/18/25', '02/25/25']

### Biweekly

This periodicity option creates bins with lengths of 2 weeks starting from `start_date`.

In [38]:
p = Period(start_date=start_date, end_date=end_date, periodicity="Biweekly")
bins = p.get_date_bins()
bins

[(datetime.date(2025, 1, 15), datetime.date(2025, 1, 28)),
 (datetime.date(2025, 1, 29), datetime.date(2025, 2, 11)),
 (datetime.date(2025, 2, 12), datetime.date(2025, 2, 25))]

In [39]:
p.get_period_labels(bins, "Biweekly")

['01/28/25', '02/11/25', '02/25/25']

### Fiscal Weeks

This periodicity option is similar to the week-grouping ISO patterns, but it doesn't strictly enforce a Monday week start. Instead, it will assume weeks start with whichever day of the week `start_date` falls on, then bin by the value or sequence given in `custom_period` (similar to how "Custom Days" works).

The following examples use a Sunday week start and create "monthly" bins of 4 weeks, 5 weeks, then 4 weeks, repeating, then create "quarterly" bins of 13 weeks each, respectively.

In [50]:
# Use a Sunday week start
start_date = datetime.date.fromisocalendar(current_year, 1, 1) - relativedelta(days=1)
end_date = datetime.date.fromisocalendar(current_year + 1, 1, 1) - relativedelta(days=2)

# "Monthly" bins grouping weeks in a 4-5-4 pattern
p = Period(start_date=start_date, end_date=end_date, periodicity="Fiscal Weeks")
bins = p.get_date_bins(custom_period=[4, 5, 4])
bins

[(datetime.date(2024, 12, 29), datetime.date(2025, 1, 25)),
 (datetime.date(2025, 1, 26), datetime.date(2025, 3, 1)),
 (datetime.date(2025, 3, 2), datetime.date(2025, 3, 29)),
 (datetime.date(2025, 3, 30), datetime.date(2025, 4, 26)),
 (datetime.date(2025, 4, 27), datetime.date(2025, 5, 31)),
 (datetime.date(2025, 6, 1), datetime.date(2025, 6, 28)),
 (datetime.date(2025, 6, 29), datetime.date(2025, 7, 26)),
 (datetime.date(2025, 7, 27), datetime.date(2025, 8, 30)),
 (datetime.date(2025, 8, 31), datetime.date(2025, 9, 27)),
 (datetime.date(2025, 9, 28), datetime.date(2025, 10, 25)),
 (datetime.date(2025, 10, 26), datetime.date(2025, 11, 29)),
 (datetime.date(2025, 11, 30), datetime.date(2025, 12, 27))]

In [51]:
p.get_period_labels(bins)

['1 (4w)-25',
 '2 (5w)-25',
 '3 (4w)-25',
 '4 (4w)-25',
 '5 (5w)-25',
 '6 (4w)-25',
 '7 (4w)-25',
 '8 (5w)-25',
 '9 (4w)-25',
 '10 (4w)-25',
 '11 (5w)-25',
 '12 (4w)-25']

In [52]:
# "Quarterly" bins of 13 weeks each
p = Period(start_date=start_date, end_date=end_date, periodicity="Fiscal Weeks")
bins = p.get_date_bins(custom_period=13)
bins

[(datetime.date(2024, 12, 29), datetime.date(2025, 3, 29)),
 (datetime.date(2025, 3, 30), datetime.date(2025, 6, 28)),
 (datetime.date(2025, 6, 29), datetime.date(2025, 9, 27)),
 (datetime.date(2025, 9, 28), datetime.date(2025, 12, 27))]

In [49]:
p.get_period_labels(bins)

['1 (13w)-25', '2 (13w)-25', '3 (13w)-25', '4 (13w)-25']

If the user does need a "stub" period at the beginning or end of the period, they can include the shorter value in the sequence, but must extend the sequence so the short value doesn't repeat. For example, the user defines months in the 4-5-4 pattern, and is two weeks into January (a 4 week month). They want to generate date bins for the rest of the year, starting from the current date. They would need to use a `custom_period` pattern of: `[2, 5, 4, 4, 5, 4, 4, 5, 4, 4, 5, 4]` to correctly generate the bins.

In [53]:
# Use a Sunday week start, but 
start_date = datetime.date.fromisocalendar(current_year, 1, 1) - relativedelta(days=1) + relativedelta(weeks=2)
end_date = datetime.date.fromisocalendar(current_year + 1, 1, 1) - relativedelta(days=2)

# "Monthly" bins grouping weeks in a 4-5-4 pattern, but with a 2-week stub period for January
p = Period(start_date=start_date, end_date=end_date, periodicity="Fiscal Weeks")
bins = p.get_date_bins(custom_period=[2, 5, 4, 4, 5, 4, 4, 5, 4, 4, 5, 4])
bins

[(datetime.date(2025, 1, 12), datetime.date(2025, 1, 25)),
 (datetime.date(2025, 1, 26), datetime.date(2025, 3, 1)),
 (datetime.date(2025, 3, 2), datetime.date(2025, 3, 29)),
 (datetime.date(2025, 3, 30), datetime.date(2025, 4, 26)),
 (datetime.date(2025, 4, 27), datetime.date(2025, 5, 31)),
 (datetime.date(2025, 6, 1), datetime.date(2025, 6, 28)),
 (datetime.date(2025, 6, 29), datetime.date(2025, 7, 26)),
 (datetime.date(2025, 7, 27), datetime.date(2025, 8, 30)),
 (datetime.date(2025, 8, 31), datetime.date(2025, 9, 27)),
 (datetime.date(2025, 9, 28), datetime.date(2025, 10, 25)),
 (datetime.date(2025, 10, 26), datetime.date(2025, 11, 29)),
 (datetime.date(2025, 11, 30), datetime.date(2025, 12, 27))]

### Calendar Month

This periodicity option creates calendar-month bins. If `start_date` doesn't fall on the first of the month, the first bin will be a stub period.

In [54]:
start_date = datetime.date(current_year, 1, 15)
end_date = datetime.date(current_year, 6, 14)

p = Period(start_date=start_date, end_date=end_date, periodicity="Calendar Month")
bins = p.get_date_bins()
bins

[(datetime.date(2025, 1, 15), datetime.date(2025, 1, 31)),
 (datetime.date(2025, 2, 1), datetime.date(2025, 2, 28)),
 (datetime.date(2025, 3, 1), datetime.date(2025, 3, 31)),
 (datetime.date(2025, 4, 1), datetime.date(2025, 4, 30)),
 (datetime.date(2025, 5, 1), datetime.date(2025, 5, 31)),
 (datetime.date(2025, 6, 1), datetime.date(2025, 6, 14))]

In [55]:
p.get_period_labels(bins, "Calendar Month")

['Jan-25', 'Feb-25', 'Mar-25', 'Apr-25', 'May-25', 'Jun-25']

### Monthly

This periodicity option creates monthly bins starting from `start_date`.

In [56]:
start_date = datetime.date(current_year, 1, 15)
end_date = datetime.date(current_year, 6, 14)

p = Period(start_date=start_date, end_date=end_date, periodicity="Monthly")
bins = p.get_date_bins()
bins

[(datetime.date(2025, 1, 15), datetime.date(2025, 2, 14)),
 (datetime.date(2025, 2, 15), datetime.date(2025, 3, 14)),
 (datetime.date(2025, 3, 15), datetime.date(2025, 4, 14)),
 (datetime.date(2025, 4, 15), datetime.date(2025, 5, 14)),
 (datetime.date(2025, 5, 15), datetime.date(2025, 6, 14))]

In [57]:
p.get_period_labels(bins, "Monthly")

['Feb-25', 'Mar-25', 'Apr-25', 'May-25', 'Jun-25']

### Calendar Quarter

This periodicity option creates quarterly bins (periods of 3 months) based on the calendar year. Quarter 1 is Jan-Mar, Q2 is Apr-Jun, Q3 is Jul-Sep, and Q4 is Oct-Dec. If `start_date` doesn't fall at the beginning of the quarter, the first bin will be a stub period.

In [58]:
start_date = datetime.date(current_year, 1, 15)
end_date = datetime.date(current_year + 1, 3, 31)

p = Period(start_date=start_date, end_date=end_date, periodicity="Calendar Quarter")
bins = p.get_date_bins()
bins

[(datetime.date(2025, 1, 15), datetime.date(2025, 3, 31)),
 (datetime.date(2025, 4, 1), datetime.date(2025, 6, 30)),
 (datetime.date(2025, 7, 1), datetime.date(2025, 9, 30)),
 (datetime.date(2025, 10, 1), datetime.date(2025, 12, 31)),
 (datetime.date(2026, 1, 1), datetime.date(2026, 3, 31))]

In [59]:
p.get_period_labels(bins, "Calendar Quarter")

['03-25Q', '06-25Q', '09-25Q', '12-25Q', '03-26Q']

### Quarterly

This periodicity option creates quarterly bins (periods of 3 months) starting from `start_date`.

In [60]:
start_date = datetime.date(current_year, 11, 15)
end_date = datetime.date(current_year + 1, 11, 14)

p = Period(start_date=start_date, end_date=end_date, periodicity="Quarterly")
bins = p.get_date_bins()
bins

[(datetime.date(2025, 11, 15), datetime.date(2026, 2, 14)),
 (datetime.date(2026, 2, 15), datetime.date(2026, 5, 14)),
 (datetime.date(2026, 5, 15), datetime.date(2026, 8, 14)),
 (datetime.date(2026, 8, 15), datetime.date(2026, 11, 14))]

In [61]:
p.get_period_labels(bins, "Quarterly")

['02-26Q', '05-26Q', '08-26Q', '11-26Q']

### Calendar Year

This periodicity option creates calendar year bins from Jan-Dec. If `start_date` doesn't fall at the beginning of the year, the first bin will be a stub period.

In [62]:
start_date = datetime.date(current_year, 1, 15)
end_date = datetime.date(current_year + 2, 12, 31)

p = Period(start_date=start_date, end_date=end_date, periodicity="Calendar Year")
bins = p.get_date_bins()
bins

[(datetime.date(2025, 1, 15), datetime.date(2025, 12, 31)),
 (datetime.date(2026, 1, 1), datetime.date(2026, 12, 31)),
 (datetime.date(2027, 1, 1), datetime.date(2027, 12, 31))]

In [63]:
p.get_period_labels(bins, "Calendar Year")

['12/31/25', '12/31/26', '12/31/27']

### Annually

This periodicity option creates yearly bins starting from `start_date`.

In [64]:
start_date = datetime.date(current_year, 7, 15)
end_date = datetime.date(current_year + 2, 7, 14)

p = Period(start_date=start_date, end_date=end_date, periodicity="Annually")
bins = p.get_date_bins()
bins

[(datetime.date(2025, 7, 15), datetime.date(2026, 7, 14)),
 (datetime.date(2026, 7, 15), datetime.date(2027, 7, 14))]

In [65]:
p.get_period_labels(bins, "Annually")

['07/14/26', '07/14/27']